In [7]:
import pandas as pd

def part_period_balancing(requirements, setup_cost, holding_cost, initial_inventory):
    periods = len(requirements)
    i = 0
    ending_inventory = 0
    total_cost = 0
    detailed_plan = []
    
    # Adjust the first period's requirement based on initial inventory
    if periods > 0:
        effective_requirement = max(0, requirements[0] - initial_inventory)
        requirements = [effective_requirement] + requirements[1:]
    
    while i < periods:
        cumulative_holding_cost = 0
        best_j = i
        
        for j in range(i, periods):
            cumulative_holding_cost += holding_cost * (j - i) * requirements[j]
            
            if cumulative_holding_cost >= setup_cost:
                break
            
            best_j = j
        
        # If cumulative holding cost exceeds setup cost, choose the best period
        if j < periods and cumulative_holding_cost > setup_cost:
            previous_holding_cost = cumulative_holding_cost - holding_cost * (j - i) * requirements[j]
            if abs(previous_holding_cost - setup_cost) < abs(cumulative_holding_cost - setup_cost):
                best_j = j - 1
                cumulative_holding_cost = previous_holding_cost
        
        total_production = sum(requirements[i:best_j + 1])
        
        if total_production > 0:
            total_cost += setup_cost  # Add setup cost once for the batch
        
        for k in range(i, best_j + 1):
            if k == i:
                production_amount = total_production
            else:
                production_amount = 0
            
            inventory_cost = ending_inventory * holding_cost
            total_cost += inventory_cost
            ending_inventory += production_amount - requirements[k]
            ending_inventory = max(0, ending_inventory)
            detailed_plan.append((k + 1, requirements[k], production_amount, ending_inventory))
        
        i = best_j + 1
    
    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [18, 30, 42, 5, 20]
setup_cost = 300
holding_cost = 3
initial_inventory = 0

print("Part Period Balancing Lot Sizing Heuristic:")
df, cost = part_period_balancing(requirements, setup_cost, holding_cost, initial_inventory)
print(df.to_string(index=False))
print("Total Cost: ", cost)


Part Period Balancing Lot Sizing Heuristic:
 Period  Requirements  Production  Ending Inventory
      1            18          48                30
      2            30           0                 0
      3            42          67                25
      4             5           0                20
      5            20           0                 0
Total Cost:  825
